In [95]:
!pip install bs4
!pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [96]:
from bs4 import BeautifulSoup as bs
import requests
from re import search
import re
import pprint

In [97]:
!apt install mongodb > log 

In [98]:
!service mongodb start

 * Starting database mongodb
   ...done.


In [99]:
from pymongo import MongoClient

In [100]:
client = MongoClient()

In [101]:
db  = client.vacancies_db
hhru_vac = db.hhru_vac


In [102]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

In [103]:
def hh_search (text):
  url = 'https://barnaul.hh.ru/search/vacancy'
#https://barnaul.hh.ru/search/vacancy?text=data+scientist&area=2&items_on_page=20&page=0
#https://barnaul.hh.ru/search/vacancy?&text=data+scientist&area=2&items_on_page=20&page=0
  params = {
    'area': 2,
  #  'search_field': 'description',
   # 'search_field': 'company_name',
    'text': text,
    'items_on_page': 20,
    'page': 0,

}
 # headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
  vacancies_list = []
  vacancy_dict = {}
  hhru_vac.drop()
  i = 0
  while True:
    response = requests.get(url=url, params=params, headers=headers)
    soup = bs(response.text, 'html.parser')
    vacancies = soup.find_all('div', {'class': 'serp-item'})
    for vacancy in vacancies:
      vacansy_name = vacancy.find('a', {'class': 'serp-item__title'}).getText()
      vacansy_link = vacancy.find('a', {'class': 'serp-item__title'})['href']
      vacansy_salary = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
      if vacansy_salary is not None:
        vacansy_salary = vacansy_salary.getText()
        if search('до', vacansy_salary):
          salary_list = re.findall(r'\d+', vacansy_salary) 
          max_salary = int(salary_list[0] + salary_list[1])
          min_salary = None
        elif search('от', vacansy_salary):
          salary_list = re.findall(r'\d+', vacansy_salary)
          min_salary = int(salary_list[0] + salary_list[1])
          max_salary = None
        elif search(' – ', vacansy_salary):
          salary_list = re.findall(r'\d+', vacansy_salary)
          if len(salary_list) < 3:
            min_salary = int(salary_list[0]) 
            max_salary = int(salary_list[1]) 
          else:
            min_salary = int(salary_list[0] + salary_list[1])
            max_salary = int(salary_list[2] + salary_list[3])
      else:
        max_salary = None
        min_salary = None
      i = i + 1
      vacancy_dict = {
        'N': i,
        'website': 'headhunter',
        'name': vacansy_name,
        #'link': vacansy_link,
        'min_salary': min_salary,
        'max_salary': max_salary,
        }
      vacancies_list.append(vacancy_dict)
      hhru_vac.insert_one(vacancy_dict)
    if len(vacancies) > 0:
      params['page'] = params['page'] + 1  
    else:
      break
  return(hhru_vac) 


In [104]:
hh_search('Data Scientist')

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'vacancies_db'), 'hhru_vac')

In [105]:
collection = hh_search('Data Scientist')

In [106]:
for doc in collection.find():
  print(doc)

{'_id': ObjectId('633953125433945b7bd02c2f'), 'N': 1, 'website': 'headhunter', 'name': 'Data Scientist', 'min_salary': None, 'max_salary': None}
{'_id': ObjectId('633953125433945b7bd02c30'), 'N': 2, 'website': 'headhunter', 'name': 'Data Scientist', 'min_salary': None, 'max_salary': None}
{'_id': ObjectId('633953125433945b7bd02c31'), 'N': 3, 'website': 'headhunter', 'name': 'Data scientist/Antifraud analyst', 'min_salary': None, 'max_salary': None}
{'_id': ObjectId('633953125433945b7bd02c32'), 'N': 4, 'website': 'headhunter', 'name': 'Риск-менеджер (Data Scientist)', 'min_salary': None, 'max_salary': None}
{'_id': ObjectId('633953125433945b7bd02c33'), 'N': 5, 'website': 'headhunter', 'name': 'Data Scientist', 'min_salary': None, 'max_salary': None}
{'_id': ObjectId('633953125433945b7bd02c34'), 'N': 6, 'website': 'headhunter', 'name': 'Data Scientist', 'min_salary': None, 'max_salary': 105000}
{'_id': ObjectId('633953125433945b7bd02c35'), 'N': 7, 'website': 'headhunter', 'name': 'Senior

In [107]:
def find_salary():
  salary = int(input('Введите минимальную зарплату для поиска: '))
  expensive_vacs = hhru_vac.find({'min_salary':{'$gte':salary}}).sort('min_salary')
  print('Вакансии с заработной платой больше ', salary)
  for item in expensive_vacs:
    print(item)
  

find_salary()

Введите минимальную зарплату для поиска: 100
Вакансии с заработной платой больше  100
{'_id': ObjectId('633953145433945b7bd02c52'), 'N': 36, 'website': 'headhunter', 'name': 'Аналитик по динамическому ценообразованию', 'min_salary': 120000, 'max_salary': None}
{'_id': ObjectId('633953125433945b7bd02c42'), 'N': 20, 'website': 'headhunter', 'name': 'Middle/Senior Data Scientist', 'min_salary': 200000, 'max_salary': 400000}


In [110]:
number_vacancies = len(list(db.hhru_vac.find()))
print("Всего %d вакансий" %number_vacancies)

Всего 37 вакансий
